In [252]:
#! /usr/bin/env python3

import pandas as pd
import numpy as np
import os

# test for forlder
if os.path.exists("../../RENAME_ME/"):
    raise KeyboardInterrupt 
os.system(
    "rsync -aP raid:/home/tgrogers-raid/a/pan251/accel-sim-framework/*.csv ../../RENAME_ME/"
)

KeyboardInterrupt: 

In [341]:
#! /usr/bin/env python3

import csv


def get_csv_data_for_merge(filepath):
    all_named_kernels = {}
    stat_map = {}
    apps = []
    cached_apps = []
    configs = []
    cached_configs = []
    stats = []
    gpgpu_build_num = None
    gpgpu_build_nums = set()
    accel_build_num = None
    accel_build_nums = set()
    data = {}
    with open(filepath, "r") as data_file:
        reader = csv.reader(data_file)  # define reader object
        state = "start"
        for row in reader:  # loop through rows in csv file
            if len(row) != 0 and row[0].startswith("----"):
                state = "find-stat"
                continue
            if state == "find-stat":
                current_stat = row[0]
                stats.append(current_stat)
                state = "find-apps"
                continue
            if state == "find-apps":
                apps = row[1:]
                state = "process-cfgs"
                continue
            if state == "process-cfgs":
                if len(row) == 0:
                    if any_data:
                        cached_apps = apps
                        cached_configs = configs
                        for config in configs:
                            count = 0
                            for appargs_kname in apps:
                                first_delimiter = appargs_kname.find("--")
                                appargs = appargs_kname[:first_delimiter]
                                kname = appargs_kname[first_delimiter + 2 :]
                                if (
                                    current_stat == "GPGPU-Sim-build"
                                    and data[config][count] != "NA"
                                ):
                                    gpgpu_build_num = data[config][count][21:28]
                                    gpgpu_build_nums.add(gpgpu_build_num)
                                if (
                                    current_stat == "Accel-Sim-build"
                                    and data[config][count] != "NA"
                                ):
                                    accel_build_num = data[config][count][16:23]
                                    accel_build_nums.add(accel_build_num)
                                stat_map[
                                    kname + appargs + config + current_stat
                                ] = data[config][count]
                                count += 1
                    apps = []
                    configs = []
                    data = {}
                    state = "start"
                    any_data = False
                    continue
                else:
                    any_data = True
                    if accel_build_num != None and gpgpu_build_num != None:
                        full_config = (
                            row[0]
                            # + "-accel-"
                            # + str(accel_build_num)
                            # + "-gpgpu-"
                            # + str(gpgpu_build_num)
                        )
                    else:
                        full_config = row[0]
                    configs.append(full_config)
                    data[full_config] = row[1:]

    app_and_args = []
    for appargs_kname in cached_apps:
        first_delimiter = appargs_kname.find("--")
        appargs = appargs_kname[:first_delimiter]
        kname = appargs_kname[first_delimiter + 2 :]
        if appargs not in all_named_kernels:
            all_named_kernels[appargs] = []
            app_and_args.append(appargs)
        all_named_kernels[appargs].append(kname)

    # The assumption here is that every entry in each stats file is run with the same
    # git hash number, if not we are just going to warn and fail.
    if len(gpgpu_build_nums) > 1 or len(accel_build_nums) > 1:
        exit(
            "File {0} contains more than one gpgpu_build_num or accelsim_build_num - this assumes one stats file has one build num: gpgpu: {1}"
            + " accel: {2}".format(filepath, gpgpu_build_nums, accel_build_nums)
        )
    return (
        all_named_kernels,
        stat_map,
        app_and_args,
        cached_configs,
        stats,
        gpgpu_build_nums,
    )

wl = 'sponza_4k'
dataset = 'data2/'
(
    all_named_kernels,
    stat_map,
    apps_and_args,
    configs,
    stats,
    gpgpu_build_nums,
) = get_csv_data_for_merge("../../" + dataset + wl + ".csv")

# workloads = ["sponza_2k", "sponza_4k"]
# workloads = ["pbrtexture_2k","pbrtexture_4k","instancing_2k","instancing_4k","render_passes_2k","render_passes_4k"]
workloads = ["pbrtexture_2k","pbrtexture_4k","instancing_2k","instancing_4k","render_passes_2k","render_passes_4k", "sponza_2k", 
             "sponza_4k", 
             "materials_2k", "materials_4k", "platformer_2k", "platformer_4k"]
wl_to_name = {
    "pbrtexture_2k": "PT2",
    "pbrtexture_4k": "PT4",
    "instancing_2k": "IT2",
    "instancing_4k": "IT4",
    "render_passes_2k": "SPL2",
    "render_passes_4k": "SPL4",
    "sponza_2k": "SPH2",
    "sponza_4k": "SPH4",
    "materials_2k": "MT2",
    "materials_4k": "MT4",
    "platformer_2k": "PL2",
    "platformer_4k": "PL4"
}
patterns = ['', '/', '\\', 'x', '-', '|', '+', '.']


apps_and_args

['sponza_4k/NO_ARGS',
 'sponza_4k/all1',
 'sponza_4k/all2',
 'sponza_4k/all3',
 'sponza_4k/ritnet',
 'sponza_4k/hotlab']

In [342]:
# Normalized Latency

import plotly
import plotly.graph_objects as go
import statistics

stats = {}
compute_cycles = {}
mesa_cycles = {}
total_cycles = {}
compute_ipcs = {}
mesa_ipcs = {}
appconfigs = []
tot_ipcs = {}
l2_misses = {}

for wl in workloads:
    (
    all_named_kernels,
    stat_map,
    apps_and_args,
    configs,
    stats,
    gpgpu_build_nums,
    )= get_csv_data_for_merge("../../" + dataset + wl + ".csv")

    for app in apps_and_args:
        for config in configs:
            mesa = 0
            max_g_cycle = 0
            max_c_cycle = 0
            l2_miss = 0
            max_c_kernel = None
            max_g_kernel = None
            for kernel in all_named_kernels[app]:
                kac = kernel + app + config
                cycle = (float(stat_map[kac + "gpu_tot_sim_cycle\\s*=\\s*(.*)"].replace("NA", "0")))
                if "MESA" in kernel:
                    if cycle > max_g_cycle:
                        max_g_cycle = cycle
                        max_g_kernel = kernel
                    l2_miss += (float(stat_map[kac + 
                            "\\s+L2_cache_stats_breakdown\\[GLOBAL_ACC_R\\]\\[MISS\\]\\s*=\\s*(.*)"]
                            .replace("NA", "0")))
                    # l2_miss += (float(stat_map[kac +
                    #         "\\s+L2_cache_stats_breakdown\\[TEXTURE_ACC_R\\]\\[MISS\\]\\s*=\\s*(.*)"]
                    #         .replace("NA", "0")))
                else:
                    if cycle > max_c_cycle:
                        max_c_cycle = cycle
                        max_c_kernel = kernel

                
            l2_misses[app + config] = l2_miss
            mesa_cycles[app + config] = max_g_cycle
            compute_cycles[app + config] = max_c_cycle
            total_cycles = max(max_g_cycle, max_c_cycle)
            # if max_g_cycle > max_c_cycle:
            #     tot_ipcs[app + config] = float(stat_map[max_g_kernel + app + config + "gpu_tot_ipc\\s*=\\s*(.*)"].replace("NA", "0"))
            # else:
            #     tot_ipcs[app + config] = float(stat_map[max_c_kernel + app + config + "gpu_tot_ipc\\s*=\\s*(.*)"].replace("NA", "0"))
            # if config == "ORIN-SASS-VISUAL" and max_c_kernel != None:
            #     tot_ipcs[app + config] = float(stat_map[max_c_kernel + app + config + "gpu_tot_ipc\\s*=\\s*(.*)"].replace("NA", "0"))
            print(app + config, 
                  f'{mesa_cycles[app + config] :,}', f'{compute_cycles[app + config]:,}', 
                #   tot_ipcs[app + config],
                # l2_misses[app + config]
                  )
                  

pbrtexture_2k/NO_ARGSORIN-SASS-VISUAL 3,949,618.0 0
pbrtexture_2k/NO_ARGSORIN-SASS-concurrent-VISUAL 3,939,330.0 0
pbrtexture_2k/NO_ARGSORIN-SASS-concurrent-fg-VISUAL 3,854,797.0 0
pbrtexture_2k/NO_ARGSORIN-SASS-concurrent-fg-invalidate_l2-best-VISUAL 3,856,700.0 0
pbrtexture_2k/NO_ARGSORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL 3,852,132.0 0
pbrtexture_2k/all1ORIN-SASS-VISUAL 3,952,963.0 5,336,775.0
pbrtexture_2k/all1ORIN-SASS-concurrent-VISUAL 4,596,164.0 2,312,353.0
pbrtexture_2k/all1ORIN-SASS-concurrent-fg-VISUAL 4,548,503.0 4,611,480.0
pbrtexture_2k/all1ORIN-SASS-concurrent-fg-invalidate_l2-best-VISUAL 4,514,319.0 4,572,385.0
pbrtexture_2k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL 4,807,174.0 4,294,949.0
pbrtexture_2k/all2ORIN-SASS-VISUAL 3,952,449.0 6,741,972.0
pbrtexture_2k/all2ORIN-SASS-concurrent-VISUAL 5,259,985.0 4,567,609.0
pbrtexture_2k/all2ORIN-SASS-concurrent-fg-VISUAL 4,550,949.0 5,991,766.0
pbrtexture_2k/all2ORIN-SASS-concurrent-fg-inval

In [472]:
cs_delay = 260000
vio1_baseline = 1383788

mesa_basline = [3962030, 5509920, 4416958, 4683252, 4065905, 8220297]
mesa_cs = [int(x + cs_delay) for x in mesa_basline]

mesa_dynamic = [4554819.0,6094342.0,4789530.0,5027274.0,4465824.0,8563620.0]
vio1_dynamic = [2312623.0,2180626.0,3027435.0,2796810.0,2457911.0,2432705.0]

mesa_ucp = [5180611, 6686349, 4961336, 5224116, 4691041, 8897084]
vio1_ucp = [3044508, 2816280, 3165919, 3067841, 2702536, 2787477]

mesa_slicer = [
    mesa_cycles["pbrtexture_2k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL"],
    mesa_cycles["pbrtexture_4k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL"],
    mesa_cycles["instancing_2k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL"],
    mesa_cycles["instancing_4k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL"],
    mesa_cycles["render_passes_2k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL"],
    mesa_cycles["render_passes_4k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL"],
]

vio1_slicer = [
    compute_cycles["pbrtexture_2k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL"],
    compute_cycles["pbrtexture_4k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL"],
    compute_cycles["instancing_2k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL"],
    compute_cycles["instancing_4k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL"],
    compute_cycles["render_passes_2k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL"],
    compute_cycles["render_passes_4k/all1ORIN-SASS-concurrent-fg-slicer-invalidate_l2-best-VISUAL"],
]

system_cs = [x + vio1_baseline for x in mesa_cs]
system_slicer = [max(x,y) for x,y in zip(mesa_slicer, vio1_slicer)]
system_ucp = [max(x,y) for x,y in zip(mesa_ucp, vio1_ucp)]
system_dynamic = [max(x,y) for x,y in zip(mesa_dynamic, vio1_dynamic)]

throughput_mesa_cs = [x/y for x,y in zip(system_cs, system_cs)]
throughput_mesa_slicer = [x/y for x,y in zip(system_cs, mesa_slicer)]
throughput_mesa_dynamic = [x/y for x,y in zip(system_cs, mesa_dynamic)]
throughput_mesa_ucp = [x/y for x,y in zip(system_cs, mesa_ucp)]

throughput_vio1_cs = [x/y for x,y in zip(system_cs, system_cs)]
throughput_vio1_slicer = [x/y for x,y in zip(system_cs, vio1_slicer)]
throughput_vio1_dynamic = [x/y for x,y in zip(system_cs, vio1_dynamic)]
throughput_vio1_ucp = [x/y for x,y in zip(system_cs, vio1_ucp)]

latency_mesa_cs = [x/y for x,y in zip(mesa_cs, mesa_basline)]
latency_mesa_slicer = [x/y for x,y in zip(mesa_slicer, mesa_basline)]
latency_mesa_dynamic = [x/y for x,y in zip(mesa_dynamic, mesa_basline)]
latency_mesa_ucp = [x/y for x,y in zip(mesa_ucp, mesa_basline)]

latency_vio1_cs = [1 for x in mesa_basline]
latency_vio1_slicer = [x/vio1_baseline for x in vio1_slicer]
latency_vio1_dynamic = [x/vio1_baseline for x in vio1_dynamic]
latency_vio1_ucp = [x/vio1_baseline for x in vio1_ucp]

throughput_mesa_cs.append(statistics.geometric_mean(throughput_mesa_cs))
throughput_mesa_slicer.append(statistics.geometric_mean(throughput_mesa_slicer))
throughput_mesa_dynamic.append(statistics.geometric_mean(throughput_mesa_dynamic))
throughput_mesa_ucp.append(statistics.geometric_mean(throughput_mesa_ucp))
throughput_vio1_cs.append(statistics.geometric_mean(throughput_vio1_cs))
throughput_vio1_slicer.append(statistics.geometric_mean(throughput_vio1_slicer))
throughput_vio1_dynamic.append(statistics.geometric_mean(throughput_vio1_dynamic))
throughput_vio1_ucp.append(statistics.geometric_mean(throughput_vio1_ucp))
latency_mesa_cs.append(statistics.geometric_mean(latency_mesa_cs))
latency_mesa_slicer.append(statistics.geometric_mean(latency_mesa_slicer))
latency_mesa_dynamic.append(statistics.geometric_mean(latency_mesa_dynamic))
latency_mesa_ucp.append(statistics.geometric_mean(latency_mesa_ucp))
latency_vio1_cs.append(statistics.geometric_mean(latency_vio1_cs))
latency_vio1_slicer.append(statistics.geometric_mean(latency_vio1_slicer))
latency_vio1_dynamic.append(statistics.geometric_mean(latency_vio1_dynamic))
latency_vio1_ucp.append(statistics.geometric_mean(latency_vio1_ucp))

latency_mesa_ucp

[1.307564808948948,
 1.2135110854604059,
 1.1232472665576625,
 1.1154889807339003,
 1.153750763975056,
 1.0823312101740363,
 1.163643554751417]

In [611]:
gr2_to_plot = []
cp2_to_plot = []
gr1_to_plot = []
cp1_to_plot = []
gr2_hotlab_to_plot = []
cp2_hotlab_to_plot = []
gr1_hotlab_to_plot = []
cp1_hotlab_to_plot = []
x_mesa_labels = []
x_vio_labels = []
x_holo_labels = []
x_nn_labels = []
x_slam_labels = []
x_names = []
x_sizes = []
font_size = 20

for wl in workloads:
    if "sponza" in wl:
        break
    x_names.append(wl.split("_")[0])
    x_sizes.append(wl.split("_")[-1])
    x_mesa_labels.append(wl_to_name[wl])
    x_vio_labels.append("VIO")
    x_holo_labels.append("HOLO")
    x_nn_labels.append("NN")
    x_slam_labels.append("SLAM")

x_mesa_labels.append("GeoMean")
x_vio_labels.append("VIO")
x_holo_labels.append("HOLO")
x_nn_labels.append("NN")
x_slam_labels.append("SLAM")

ones = [0 for i in range(len(x_mesa_labels))]
fig0 = go.Figure()
fig0.add_trace(
    go.Scatter(
        name="TAP",
        x=[
            x_mesa_labels + x_mesa_labels,
            ["TAP" for i in throughput_mesa_ucp] + ["TAP" for i in throughput_vio1_ucp],
        ],
        y=throughput_mesa_ucp + throughput_vio1_ucp,
        mode="markers",
        marker_symbol=["x" for i in throughput_mesa_ucp] + ["square" for i in throughput_vio1_ucp],
        marker_size=12,
        showlegend=False
    )
)
fig0.add_trace(
    go.Scatter(
        name="Warp-Slicer",
        x=[
            x_mesa_labels + x_mesa_labels,
            ["Warp Slicer" for i in throughput_mesa_slicer] + ["Warp Slicer" for i in throughput_vio1_slicer],
        ],
        y=throughput_mesa_slicer + throughput_vio1_slicer,
        mode="markers",
        marker_symbol=["x" for i in throughput_mesa_slicer] + ["square" for i in throughput_vio1_slicer],
        marker_size=12,
        showlegend=False
    )
)
fig0.add_trace(
    go.Scatter(
        name="Render",
        x=[
            x_mesa_labels + x_mesa_labels,
            ["Split Reality" for i in throughput_mesa_dynamic],
        ],
        y=throughput_mesa_dynamic + throughput_vio1_dynamic,
        mode="markers+text",
        marker_symbol=["x" for i in throughput_mesa_dynamic],
        marker_size=12,
        marker_color="green",
        # 2 digit
        text = [f'{x:.2f}' for x in throughput_mesa_dynamic + throughput_vio1_dynamic],
        textposition="bottom center",
        textfont=dict(size=font_size),
    )
)
fig0.add_trace(
    go.Scatter(
        name="VIO",
        x=[
            x_mesa_labels,
            ["Split Reality" for i in throughput_vio1_dynamic],
        ],
        y=throughput_vio1_dynamic,
        mode="markers+text",
        marker_symbol=["square" for i in throughput_vio1_dynamic],
        marker_size=12,
        marker_color="green",
        text = [f'{x:.2f}' for x in throughput_vio1_dynamic],
        textposition="top center",
        textfont=dict(size=font_size),
    )
)

# change width and height of the figure
fig0.update_layout(
    autosize=False,
    width=1000,
    height=500,
    # title="Normalized Latency",
)
# add vertical line between groups
for i in range(len(x_mesa_labels)):
    fig0.add_shape(
        type="line",
        x0=2.5 + i * 3,
        y0=0,
        x1=2.5 + i * 3,
        y1=7.2,
        line=dict(
            color="gainsboro",
            width=2,
        ),
        layer="below"
    )
# change x axis title
fig0.update_layout(xaxis_title="Workloads", yaxis_title="Throughput")
# change y axis range
fig0.update_yaxes(range=[0, 5.2], dtick=1)
fig0.update_xaxes(range=[-0.5,20.5], dtick=1)
# fig0.update_xaxes(dtick=5)

# remove background
fig0.update_layout(plot_bgcolor='rgba(0,0,0,0)')
# show y line
fig0.update_layout(yaxis=dict(showline=True, linewidth=2, linecolor='black'))
fig0.update_layout(xaxis=dict(showline=True, linewidth=2, linecolor='black'))
# show grid
fig0.update_layout(yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='gainsboro'))
# change font size
fig0.update_layout(yaxis=dict(tickfont=dict(size=font_size)))
fig0.update_layout(xaxis=dict(tickfont=dict(size=font_size)))
fig0.update_layout(title_font_size=font_size)
fig0.update_layout(legend=dict(font=dict(size=font_size)))
fig0.update_layout(xaxis_title_font_size=font_size)
fig0.update_layout(yaxis_title_font_size=font_size)
# change legend position to top
fig0.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0.4,
        
    )
)
# change margin
fig0.update_layout(margin=dict(l=50, r=50, t=20, b=100))

fig1 = go.Figure()
fig1.add_trace(
    go.Scatter(
        name="TAP",
        x=[
            x_mesa_labels + x_mesa_labels,
            ["TAP" for i in latency_mesa_ucp] + ["TAP" for i in latency_vio1_ucp],
        ],
        y=latency_mesa_ucp + latency_vio1_ucp,
        mode="markers",
        marker_symbol=["x" for i in latency_mesa_ucp] + ["square" for i in latency_vio1_ucp],
        marker_size=12,
        showlegend=False
    )
)
fig1.add_trace(
    go.Scatter(
        name="Warp-Slicer",
        x=[
            x_mesa_labels + x_mesa_labels,
            ["Warp Slicer" for i in latency_mesa_slicer] + ["Warp Slicer" for i in latency_vio1_slicer],
        ],
        y=latency_mesa_slicer + latency_vio1_slicer,
        mode="markers",
        marker_symbol=["x" for i in latency_mesa_slicer] + ["square" for i in latency_vio1_slicer],
        marker_size=12,
        showlegend=False
    )
)
fig1.add_trace(
    go.Scatter(
        name="Render",
        x=[
            x_mesa_labels + x_mesa_labels,
            ["Split Reality" for i in latency_mesa_dynamic],
        ],
        y=latency_mesa_dynamic + latency_vio1_dynamic,
        mode="markers+text",
        marker_symbol=["x" for i in latency_mesa_dynamic],
        marker_size=12,
        marker_color="green",
        # 2 digit
        text = [f'{x:.2f}' for x in latency_mesa_dynamic + latency_vio1_dynamic],
        textposition="bottom center",
        textfont=dict(size=font_size),
    )
)
fig1.add_trace(
    go.Scatter(
        name="VIO",
        x=[
            x_mesa_labels,
            ["Split Reality" for i in latency_vio1_dynamic],
        ],
        y=latency_vio1_dynamic,
        mode="markers+text",
        marker_symbol=["square" for i in latency_vio1_dynamic],
        marker_size=12,
        marker_color="green",
        text = [f'{x:.2f}' for x in latency_vio1_dynamic],
        textposition="top center",
        textfont=dict(size=font_size),
    )
)

# change width and height of the figure
fig1.update_layout(
    autosize=False,
    width=1000,
    height=500,
    # title="Normalized Latency",
)
# add vertical line between groups
for i in range(len(x_mesa_labels)):
    fig1.add_shape(
        type="line",
        x0=2.5 + i * 3,
        y0=0,
        x1=2.5 + i * 3,
        y1=7.2,
        line=dict(
            color="gainsboro",
            width=2,
        ),
        layer="below"
    )
# change x axis title
fig1.update_layout(xaxis_title="Workloads", yaxis_title="Latency")
# change y axis range
fig1.update_yaxes(range=[0, 7.2], dtick=1)
fig1.update_xaxes(range=[-0.5,20.5], dtick=1)
# fig1.update_xaxes(dtick=5)

# remove background
fig1.update_layout(plot_bgcolor='rgba(0,0,0,0)')
# show y line
fig1.update_layout(yaxis=dict(showline=True, linewidth=2, linecolor='black'))
fig1.update_layout(xaxis=dict(showline=True, linewidth=2, linecolor='black'))
# show grid
fig1.update_layout(yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='gainsboro'))
# change font size
fig1.update_layout(yaxis=dict(tickfont=dict(size=font_size)))
fig1.update_layout(xaxis=dict(tickfont=dict(size=font_size)))
fig1.update_layout(title_font_size=font_size)
fig1.update_layout(legend=dict(font=dict(size=font_size)))
fig1.update_layout(xaxis_title_font_size=font_size)
fig1.update_layout(yaxis_title_font_size=font_size)
# change legend position to top
fig1.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0.4,
        
    )
)
# change margin
fig1.update_layout(margin=dict(l=50, r=50, t=20, b=100))
# hide legend
# fig1.update_layout(showlegend=False)
# add custom legend
fig0.show()
fig1.show()

plotly.io.write_image(fig0, "/Users/pan/Documents/GitHub/ar_vr_gpu_overleaf/Figures/overall_throughput.pdf", format="pdf")
plotly.io.write_image(fig1, "/Users/pan/Documents/GitHub/ar_vr_gpu_overleaf/Figures/overall_latency.pdf", format="pdf")

In [24]:
stats = {}
appconfigs = []

ipc1_to_plot = []
ipc2_to_plot = []
ipc1_holo_to_plot = []
ipc2_holo_to_plot = []
for wl in workloads:
    x_names.append(wl.split("_")[0])
    x_sizes.append(wl.split("_")[-1])
    app = wl + "/all4"
    config = app + "ORIN-SASS-concurrent-fg-invalidate_l2-best-VISUAL"
    baseline = app + "ORIN-SASS-VISUAL"
    ipc1_to_plot.append(tot_ipcs[config] / (tot_ipcs[baseline]))
    print(tot_ipcs[config] / (tot_ipcs[baseline]))

    config =app+"ORIN-SASS-concurrent-fg-slicer-VISUAL"
    ipc2_to_plot.append(tot_ipcs[config] / (tot_ipcs[baseline]))

    app = wl + "/hotlab"
    config = app + "ORIN-SASS-concurrent-fg-invalidate_l2-best-VISUAL"
    baseline = app + "ORIN-SASS-VISUAL"
    ipc1_holo_to_plot.append(tot_ipcs[config] / (tot_ipcs[baseline]))

    config = app+"ORIN-SASS-concurrent-fg-slicer-VISUAL"
    ipc2_holo_to_plot.append(tot_ipcs[config] / (tot_ipcs[baseline]))




fig = go.Figure()
fig.add_trace(
    go.Bar(
        name="Warp Slicer",
        x=[
            x_mesa_labels + x_mesa_labels + x_mesa_labels + x_mesa_labels,
            x_vio_labels
            + x_holo_labels + x_nn_labels + x_slam_labels,
        ],
        y=ipc2_to_plot + ipc2_holo_to_plot + ones + ones,
        # marker=dict(color=colors['Frame - cache']),
        marker_pattern_shape=patterns["Frame - cache"],
    )
)
fig.add_trace(
    go.Bar(
        name="Our Design",
        x=[
            x_mesa_labels + x_mesa_labels + x_mesa_labels + x_mesa_labels,
            x_vio_labels
            + x_holo_labels + x_nn_labels + x_slam_labels,
        ],
        y=ipc1_to_plot + ipc1_holo_to_plot + ones + ones,
        # marker=dict(color=colors['Frame - cache + scheduler']),
        marker_pattern_shape=patterns["Frame - cache + scheduler"],
    )
)

# change width and height of the figure
fig.update_layout(
    autosize=False,
    width=1200,
    height=400,
    # title="Normalized Latency",
)
# Change the bar mode
fig.update_layout(barmode="group")
# add title in middle
fig.update_layout(
    title={
        "text": "Normalized IPCs",
        "y": 0.97,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    }
)
# change bar gap
fig.update_layout(bargap=0.4, bargroupgap=0)
# change x axis title
fig.update_layout(xaxis_title="Workloads", yaxis_title="IPC")
# change y axis range
# fig.update_yaxes(range=[0, 1.5], dtick=1)
# change y tick step
# change font size
fig.update_layout(xaxis=dict(tickfont=dict(size=10)))
fig.update_layout(yaxis=dict(tickfont=dict(size=20)))
fig.update_layout(title_font_size=20)
fig.update_layout(legend=dict(font=dict(size=15)))
fig.update_layout(xaxis_title_font_size=20)
fig.update_layout(yaxis_title_font_size=20)
# change legend position to top
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0.1,
    )
)
# change margin
fig.update_layout(margin=dict(l=0, r=10, t=100, b=100))

fig.show()

plotly.io.write_image(fig, "output_file.pdf", format="pdf")
plotly.io.write_image(fig, "tmp.png", format="png") 

1.0873113697674646
1.0693366302881888
1.0973571203587227
1.11064199035398
1.148347056008407
1.0923441957014721
1.1728478041017063
1.2636578925504733
1.2153135963642163
2.0423312462715186
1.6963949922642474


In [65]:
# MPKI

stats = {}
appconfigs = []

l2_miss_to_plot = []
l2_miss_holo_to_plot = []
l2_miss_ritnet_to_plot = []


for wl in workloads:
    x_names.append(wl.split("_")[0])
    x_sizes.append(wl.split("_")[-1])
    app = wl + "/all4"
    config = app + "ORIN-SASS-concurrent-fg-VISUAL"
    baseline = app + "ORIN-SASS-VISUAL"
    l2_miss_to_plot.append(l2_misses[config] / (l2_misses[baseline]))
    print(app, l2_misses[config], l2_misses[baseline])

    # config =app+"ORIN-SASS-concurrent-fg-slicer-VISUAL"
    # ipc2_to_plot.append(l2_misses[config] / (l2_misses[baseline]))

    app = wl + "/hotlab"
    config = app + "ORIN-SASS-concurrent-fg-VISUAL"
    baseline = app + "ORIN-SASS-VISUAL"
    l2_miss_holo_to_plot.append(l2_misses[config] / (l2_misses[baseline]))

    app = wl + "/ritnet"
    config = app + "ORIN-SASS-concurrent-fg-VISUAL"
    baseline = app + "ORIN-SASS-VISUAL"
    l2_miss_ritnet_to_plot.append(l2_misses[config] / (l2_misses[baseline]))


fig = go.Figure()
fig.add_trace(
    go.Bar(
        name="Fine-grained Concurrent",
        x=[
            x_mesa_labels + x_mesa_labels + x_mesa_labels + x_mesa_labels,
            x_vio_labels
            + x_holo_labels + x_nn_labels + x_slam_labels,
        ],
        y=l2_miss_to_plot + l2_miss_holo_to_plot + l2_miss_ritnet_to_plot + ones,
        # marker=dict(color=colors['Frame - cache']),
        marker_pattern_shape=patterns["Frame - cache"],
    )
)

# change width and height of the figure
# fig.update_layout(
#     autosize=False,
#     width=400,
#     height=300,
#     # title="Normalized Latency",
# )
# Change the bar mode
fig.update_layout(barmode="group")
# add title in middle
fig.update_layout(
    title={
        "text": "Normalized L2 Misses",
        "y": 0.97,
        "x": 0.5,
        "xanchor": "center",
        "yanchor": "top",
    }
)
# change bar gap
fig.update_layout(bargap=0.4, bargroupgap=0)
# change x axis title
fig.update_layout(xaxis_title="Workloads", yaxis_title="Normalized L2 Misses")
# change y axis range
fig.update_yaxes(range=[0, 4], dtick=1)
# change y tick step
# change font size
fig.update_layout(xaxis=dict(tickfont=dict(size=10)))
fig.update_layout(yaxis=dict(tickfont=dict(size=20)))
fig.update_layout(title_font_size=20)
fig.update_layout(legend=dict(font=dict(size=15)))
fig.update_layout(xaxis_title_font_size=20)
fig.update_layout(yaxis_title_font_size=20)
# change legend position to top
fig.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0.1,
    )
)
# change margin
fig.update_layout(margin=dict(l=0, r=10, t=100, b=100))

fig.show()

plotly.io.write_image(fig, "output_file.pdf", format="pdf")
plotly.io.write_image(fig, "tmp.png", format="png") 



pbrtexture_2k/hotlab 692493.0 692493.0
pbrtexture_4k/hotlab 692493.0 692494.0
instancing_2k/hotlab 692493.0 692493.0
instancing_4k/hotlab 692493.0 692493.0
render_passes_2k/hotlab 692493.0 692493.0
render_passes_4k/hotlab 692494.0 692493.0
sponza_2k/hotlab 692555.0 692493.0
materials_2k/hotlab 692493.0 692493.0
materials_4k/hotlab 692553.0 692493.0
platformer_2k/hotlab 692494.0 692493.0
platformer_4k/hotlab 692555.0 692494.0


In [415]:
# GPU vs CPU speedup and energy

klt_gpu_ms = 19840.6
klt_gpu_mw = 4335
klt_cpu_ms = 64783
klt_cpu_mw = 1506

harris_gpu_ms = 12990.3
harris_gpu_mw = 3912
harris_cpu_ms = 81832.3
harris_cpu_mw = 1218

optflow_gpu_ms = 16725.2
optflow_gpu_mw = 3579
optflow_cpu_ms = 49963.5
optflow_cpu_mw = 1208

distort_gpu_ms = 19548
distort_gpu_mw = 798
distort_cpu_ms = 56640.8
distort_cpu_mw = 1590

template_gpu_ms = 69620.7
template_gpu_mw = 3982
template_cpu_ms = 379611.8
template_cpu_mw = 1970

klt_gpu_mj = klt_gpu_ms * klt_gpu_mw / 1000 / 10000
klt_cpu_mj = klt_cpu_ms * klt_cpu_mw / 1000 / 10000
harris_gpu_mj = harris_gpu_ms * harris_gpu_mw / 1000 / 10000
harris_cpu_mj = harris_cpu_ms * harris_cpu_mw / 1000 / 10000
optflow_gpu_mj = optflow_gpu_ms * optflow_gpu_mw / 1000 / 10000
optflow_cpu_mj = optflow_cpu_ms * optflow_cpu_mw / 1000 / 10000
distort_gpu_mj = distort_gpu_ms * distort_gpu_mw / 1000 / 10000
distort_cpu_mj = distort_cpu_ms * distort_cpu_mw / 1000 / 10000
template_gpu_mj = template_gpu_ms * template_gpu_mw / 1000 / 10000
template_cpu_mj = template_cpu_ms * template_cpu_mw / 1000 / 10000

print(klt_gpu_mj, klt_cpu_mj)
print(harris_gpu_mj, harris_cpu_mj)
print(optflow_gpu_mj, optflow_cpu_mj)
print(distort_gpu_mj, distort_cpu_mj)
print(template_gpu_mj, template_cpu_mj)

klt_speedup = klt_cpu_ms / klt_gpu_ms
harris_speedup = harris_cpu_ms / harris_gpu_ms
optflow_speedup = optflow_cpu_ms / optflow_gpu_ms
distort_speedup = distort_cpu_ms / distort_gpu_ms
template_speedup = template_cpu_ms / template_gpu_ms

klt_energy = klt_gpu_mj / klt_cpu_mj
harris_energy = harris_gpu_mj / harris_cpu_mj
optflow_energy = optflow_gpu_mj / optflow_cpu_mj
distort_energy = distort_gpu_mj / distort_cpu_mj
template_energy = template_gpu_mj / template_cpu_mj
print(klt_energy, harris_energy, optflow_energy, distort_energy, template_energy)

geomean_speedup = (klt_speedup * harris_speedup * optflow_speedup * distort_speedup * template_speedup) ** (1/5)
geomean_energy = (klt_energy * harris_energy * optflow_energy * distort_energy * template_energy) ** (1/5)

x_labels = ["KLT Tracker", "Harris</br></br>Corner", "Optical</br></br>Flow", "Distortion", 
            "Template</br></br>Matching", "Geomean"]
speedups_to_plot = [klt_speedup, harris_speedup, optflow_speedup, distort_speedup, template_speedup, geomean_speedup]
energys_to_plot = [klt_energy, harris_energy, optflow_energy, distort_energy, template_energy, geomean_energy]

fig0 = go.Figure()
fig0.add_trace(
    go.Bar(
        name="CPU",
        x=x_labels,
        y=[1, 1, 1, 1, 1, 1],
        marker_pattern_shape=patterns[0],
    )
)
fig0.add_trace(
    go.Bar(
        name="GPU",
        x=x_labels,
        y=speedups_to_plot,
        marker_pattern_shape=patterns[1],
        text = [f'{speedup:.2f}' for speedup in speedups_to_plot],
        textposition='outside',
        textfont=dict(size=20),
    )
)

# change width and height of the figure
fig0.update_layout(
    autosize=False,
    width=800,
    height=250,
    # title="Normalized Latency",
)
# Change the bar mode
fig0.update_layout(barmode="group")
# add title in middle
# fig.update_layout(
#     title={
#         "text": "Normalized L2 Misses",
#         "y": 0.97,
#         "x": 0.5,
#         "xanchor": "center",
#         "yanchor": "top",
#     }
# )
# change bar gap
fig0.update_layout(bargap=0.2, bargroupgap=0)
# change x axis title
fig0.update_layout(
    # xaxis_title="Workloads", 
    yaxis_title="Speedup")
# change y axis range
fig0.update_yaxes(range=[0, 9], dtick=2)
# change y tick step
# change font size
fig0.update_layout(xaxis=dict(tickfont=dict(size=20)))
fig0.update_layout(yaxis=dict(tickfont=dict(size=20)))
fig0.update_layout(title_font_size=20)
fig0.update_layout(legend=dict(font=dict(size=20)))
fig0.update_layout(xaxis_title_font_size=20)
fig0.update_layout(yaxis_title_font_size=20)
# remove background
fig0.update_layout(plot_bgcolor='rgba(0,0,0,0)')
# show y line
fig0.update_layout(yaxis=dict(showline=True, linewidth=2, linecolor='black'))
fig0.update_layout(xaxis=dict(showline=True, linewidth=2, linecolor='black'))
# show grid
fig0.update_layout(yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='gainsboro'))
# change legend position to top
fig0.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0.4,
    )
)
# change margin
fig0.update_layout(margin=dict(l=0, r=10, t=0, b=60))

fig1 = go.Figure()
fig1.add_trace(
    go.Bar(
        name="CPU",
        x=x_labels,
        y=[1, 1, 1, 1, 1, 1],
        marker_pattern_shape=patterns[0],
    )
)
fig1.add_trace(
    go.Bar(
        name="GPU",
        x=x_labels,
        y=energys_to_plot,
        marker_pattern_shape=patterns[1],
        text = [f'{energy:.2f}' for energy in energys_to_plot],
        textposition='outside',
        textfont=dict(size=20),
    )
)

# for energy in energys_to_plot:
#     fig1.add_annotation(
#         x=energys_to_plot.index(energy) + 0.2,
#         y=energy + 0.25,
#         text=f'{energy:.2f}',
#         showarrow=False,
#         font=dict(size=20),
#     )

# change width and height of the figure
fig1.update_layout(
    autosize=False,
    width=800,
    height=250,
    # title="Normalized Latency",
)
# Change the bar mode
fig1.update_layout(barmode="group")
# add title in middle
# fig.update_layout(
#     title={
#         "text": "Normalized L2 Misses",
#         "y": 0.97,
#         "x": 0.5,
#         "xanchor": "center",
#         "yanchor": "top",
#     }
# )
# change bar gap
fig1.update_layout(bargap=0.2, bargroupgap=0)
# change x axis title
fig1.update_layout(
    # xaxis_title="Workloads", 
    yaxis_title="Energy")
# change y axis range
fig1.update_yaxes(range=[0, 1.52], dtick=0.5)
# change y tick step
# change font size
fig1.update_layout(xaxis=dict(tickfont=dict(size=20)))
fig1.update_layout(yaxis=dict(tickfont=dict(size=20)))
fig1.update_layout(title_font_size=20)
fig1.update_layout(legend=dict(font=dict(size=20)))
fig1.update_layout(xaxis_title_font_size=20)
fig1.update_layout(yaxis_title_font_size=20)
# remove background
fig1.update_layout(plot_bgcolor='rgba(0,0,0,0)')
# change legend position to top
fig1.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0.4,
    )
)
# change margin
fig1.update_layout(margin=dict(l=0, r=10, t=0, b=60))
# show y line
fig1.update_layout(yaxis=dict(showline=True, linewidth=2, linecolor='black'))
fig1.update_layout(xaxis=dict(showline=True, linewidth=2, linecolor='black'))
# show grid
fig1.update_layout(yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='gainsboro'))
# display x label in multiple lines

fig0.show()
fig1.show()

plotly.io.write_image(fig0, "/Users/pan/Documents/GitHub/ar_vr_gpu_overleaf/Figures/Power15WSpeedup.pdf", format="pdf")
plotly.io.write_image(fig1, "/Users/pan/Documents/GitHub/ar_vr_gpu_overleaf/Figures/Power15WEnergy.pdf", format="pdf")

8.6009001 9.7563198
5.081805359999999 9.96717414
5.985949080000001 6.0355908000000005
1.5599304 9.0058872
27.72296274 74.7835246
0.8815721784765604 0.5098541761807724 0.9917751680581129 0.17321229606340174 0.37070949635342537


In [437]:
mesa8_baseline = [1319490, 2751357, 629809, 780998, 4047467, 8198723]
mesa8_mps = [2699333, 5386902, 1161189, 1392277, 6965238, 16078866]
mesa8_mig = [2572982,5429124,1023597,1310536,7266966,15441682]
mesa8_fg = [2049977,4258278,730038,986298,5010170,10320821]

mesa8_baseline_speedup = [x/y for x,y in zip(mesa8_baseline, mesa8_baseline)]
mesa8_mps_speedup = [x/y for x,y in zip(mesa8_mps, mesa8_baseline)]
mesa8_mig_speedup = [x/y for x,y in zip(mesa8_mig, mesa8_baseline)]
mesa8_fg_speedup = [x/y for x,y in zip(mesa8_fg, mesa8_baseline)]

x_mesa_labels = []
x_vio_labels = []
x_holo_labels = []
x_nn_labels = []
x_slam_labels = []
x_names = []
x_sizes = []
font_size = 25

for wl in workloads:
    if "sponza" in wl:
        break
    x_names.append(wl.split("_")[0])
    x_sizes.append(wl.split("_")[-1])
    x_mesa_labels.append(wl_to_name[wl])
    x_vio_labels.append("VIO")
    x_holo_labels.append("HOLO")
    x_nn_labels.append("NN")
    x_slam_labels.append("SLAM")

x_mesa_labels.append("GeoMean")
x_vio_labels.append("VIO")
x_holo_labels.append("HOLO")
x_nn_labels.append("NN")
x_slam_labels.append("SLAM")

fig0 = go.Figure()
fig0.add_trace(
    go.Bar(
        name="Rendering<br>Only",
        x=x_mesa_labels,
        y=mesa8_baseline_speedup,
        marker_pattern_shape=patterns[0],
    )
)
fig0.add_trace(
    go.Bar(
        name="SM<br>Only",
        x=x_mesa_labels,
        y=mesa8_mps_speedup,
        marker_pattern_shape=patterns[1],
    )
)
fig0.add_trace(
    go.Bar(
        name="Full<br>Chip",
        x=x_mesa_labels,
        y=mesa8_mig_speedup,
        marker_pattern_shape=patterns[2],
    )
)
fig0.add_trace(
    go.Bar(
        name="Fine<br>Grained",
        x=x_mesa_labels,
        y=mesa8_fg_speedup,
        marker_pattern_shape=patterns[3],
    )
)

# change width and height of the figure
fig0.update_layout(
    autosize=False,
    width=700,
    height=250,
    # title="Normalized Latency",
)
# Change the bar mode
fig0.update_layout(barmode="group")
# add title in middle
# fig.update_layout(
#     title={
#         "text": "Normalized L2 Misses",
#         "y": 0.97,
#         "x": 0.5,
#         "xanchor": "center",
#         "yanchor": "top",
#     }
# )
# change bar gap
fig0.update_layout(bargap=0.2, bargroupgap=0)
# change x axis title
fig0.update_layout(
    xaxis_title="Workloads", 
    yaxis_title="Latency")
# change y axis range
fig0.update_yaxes(range=[0, 2.6], dtick=1)
# change y tick step
# change font size
fig0.update_layout(xaxis=dict(tickfont=dict(size=font_size)))
fig0.update_layout(yaxis=dict(tickfont=dict(size=font_size)))
fig0.update_layout(title_font_size=font_size)
fig0.update_layout(legend=dict(font=dict(size=font_size)))
fig0.update_layout(xaxis_title_font_size=font_size)
fig0.update_layout(yaxis_title_font_size=font_size)
# remove background
fig0.update_layout(plot_bgcolor='rgba(0,0,0,0)')
# show y line
fig0.update_layout(yaxis=dict(showline=True, linewidth=2, linecolor='black'))
fig0.update_layout(xaxis=dict(showline=True, linewidth=2, linecolor='black'))
# show grid
fig0.update_layout(yaxis=dict(showgrid=True, gridwidth=0.5, gridcolor='gainsboro'))
# change legend position to top
fig0.update_layout(
    legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1,
        xanchor="left",
        x=0.05,
    )
)
# change margin
fig0.update_layout(margin=dict(l=0, r=10, t=0, b=60))

fig0.show()
plotly.io.write_image(fig0, "/Users/pan/Documents/GitHub/ar_vr_gpu_overleaf/Figures/baseline_partition.pdf", format="pdf")
